# Analysis of 220k Monster job postings

 - data used is from Kaggle
 - uses skills-ml pipeline
 - trains embedding model
 - trains occupation classifier

# House-keeping

In [1]:
# general
import os
import json
import copy
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import logging
from retrying import Retrying
from io import BytesIO
from itertools import chain, islice
from typing import Dict, Text, Any, Generator
import pickle

# common schema
from skills_ml.job_postings.raw.virginia import VirginiaTransformer

# embedding
from skills_ml.algorithms.embedding.models import Word2VecModel, FastTextModel
from skills_ml.job_postings.corpora import Word2VecGensimCorpusCreator, Doc2VecGensimCorpusCreator
from skills_ml.algorithms.preprocessing import IterablePipeline
from skills_ml.algorithms import nlp
from functools import partial
from skills_ml.algorithms.embedding.train import EmbeddingTrainer

# occupation classifier
from skills_ml.storage import FSStore, open_sesame
from skills_ml.ontologies.onet import Onet
from skills_ml.algorithms.occupation_classifiers import FullSOC, SOCMajorGroup
from skills_ml.utils import itershuffle
from skills_ml.algorithms.occupation_classifiers import DesignMatrix
from skills_ml.algorithms.occupation_classifiers.train import OccupationClassifierTrainer
from skills_ml.algorithms.occupation_classifiers.test import OccupationClassifierTester
from skills_ml.evaluation.occ_cls_evaluator import OnetOccupationClassificationEvaluator
from skills_ml.algorithms.occupation_classifiers.classifiers import CombinedClassifier
from skills_ml.algorithms.embedding.train import Reiterable
from skills_ml.job_postings.corpora import SimpleCorpusCreator
from skills_ml.job_postings.corpora import Doc2VecGensimCorpusCreator, CorpusCreator

# skill extraction
from skills_ml.algorithms.skill_extractors import SkillEndingPatternExtractor
from skills_ml.job_postings.common_schema import JobPostingCollectionSample
from skills_ml.algorithms.skill_extractors import ExactMatchSkillExtractor
from skills_ml.algorithms.skill_extractors import SocScopedExactMatchSkillExtractor


/Users/oren/opt/anaconda3/envs/skills_ml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/oren/opt/anaconda3/envs/skills_ml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/oren/opt/anaconda3/envs/skills_ml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/oren/opt/anaconda3/en

# Load job posting data

In [16]:
file_name = 'indeed_finance_energy.csv' 
monster_220k = pd.read_csv(file_name).reset_index(drop=True)
monster_220k.head()

,country,country_code,date_added,has_expired,job_board,job_description,job_title,job_type,location,organization,page_url,salary,sector,uniq_id
0,United States of America,US,NaN,No,jobs.monster.com,TeamSoft is seeing an IT Support Specialist to...,IT Support Technician Job in Madison,Full Time Employee,"Madison, WI 53702",NaN,http://jobview.monster.com/it-support-technici...,NaN,IT/Software Development,11d599f229a80023d2f40e7c52cd941e
1,United States of America,US,NaN,No,jobs.monster.com,The Wisconsin State Journal is seeking a flexi...,Business Reporter/Editor Job in Madison,Full Time,"Madison, WI 53708",Printing and Publishing,http://jobview.monster.com/business-reporter-e...,NaN,NaN,e4cbb126dabf22159aff90223243ff2a
2,United States of America,US,NaN,No,jobs.monster.com,Report this job About the Job DePuy Synthes Co...,Johnson & Johnson Family of Companies Job Appl...,"Full Time, Employee",DePuy Synthes Companies is a member of Johnson...,Personal and Household Services,http://jobview.monster.com/senior-training-lea...,NaN,NaN,839106b353877fa3d896ffb9c1fe01c0
3,United States of America,US,NaN,No,jobs.monster.com,Why Join Altec? If you’re considering a career...,Engineer - Quality Job in Dixon,Full Time,"Dixon, CA",Altec Industries,http://jobview.monster.com/engineer-quality-jo...,NaN,Experienced (Non-Manager),58435fcab804439efdcaa7ecca0fd783
4,United States of America,US,NaN,No,jobs.monster.com,Position ID# 76162 # Positions 1 State CT C...,Shift Supervisor - Part-Time Job in Camphill,Full Time Employee,"Camphill, PA",Retail,http://jobview.monster.com/shift-supervisor-pa...,NaN,Project/Program Management,64d0272dc8496abfd9523a8df63c184c


# Parse into job posting schema

In [17]:
fname = '50_sample.json'
f = open(fname, 'r')
data = f.read().split('\n')[0]
schema = json.loads(data)
s50 = f.read().split('\n')[0:50]

In [18]:
df_json = []
for i in range(monster_220k.shape[0]):
    occupation = copy.deepcopy(schema)
    occupation['hiringOrganization']['location'] = monster_220k['location'][i]
    occupation['title'] = monster_220k['job_title'][i]
    occupation['jobDescription'] = monster_220k['job_description'][i].split('\n')
    df_json.append(occupation)

In [1]:
with open("monster_220k.json", "w") as write_file:
    for i in df_json:
        json.dump(i, write_file)
        write_file.write('\n')

NameError: name 'df_json' is not defined

In [19]:
## reads json data file

JobPostingType = Dict[Text, Any]
JobPostingGeneratorType = Generator[JobPostingType, None, None]
MetadataType = Dict[Text, Dict[Text, Any]]

class monsterParser(object):
    def __init__(self):
        fname = 'monster_220k.json'
        f = open(fname, 'r')
        self.lines = f.read().split('\n')
        self.transformer = VirginiaTransformer(partner_id = 'VA')
        
    def __iter__(self) -> JobPostingGeneratorType:
        for line in self.lines:
            if line:
                yield self.transformer._transform(json.loads(line))
    
    @property
    def metadata(self) -> MetadataType:
        return {'job postings': {
            'downloaded_from': 'jobs.monster.com',
            'month': '2020-08',
            'purpose': 'monster_analysis'
        }}


In [20]:
job_postings = monsterParser()

# Embedding model

In [21]:
cbow = Word2VecModel(size=200, sg=0, window=7, iter=3, batch_words=1000)
skip_gram = Word2VecModel(size=200, sg=1, window=7, iter=3, batch_words=1000)
fasttext = FastTextModel(size=200, window=7, iter=3, batch_words=1000) 

w2v_corpus_generator = Word2VecGensimCorpusCreator(job_postings)

In [22]:
trainer = EmbeddingTrainer(cbow, skip_gram, fasttext, batch_size=100)
trainer.train(w2v_corpus_generator)

2020-08-02 00:47:41,277 : INFO : Training batch #0 
2020-08-02 00:47:41,278 : INFO : collecting all words and their counts
2020-08-02 00:47:41,279 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-08-02 00:47:41,281 : INFO : collected 879 word types from a corpus of 14733 raw words and 100 sentences
2020-08-02 00:47:41,282 : INFO : Loading a fresh vocabulary
2020-08-02 00:47:41,283 : INFO : effective_min_count=5 retains 169 unique words (19% of original 879, drops 710)
2020-08-02 00:47:41,284 : INFO : effective_min_count=5 leaves 13666 word corpus (92% of original 14733, drops 1067)
2020-08-02 00:47:41,286 : INFO : deleting the raw counts dictionary of 879 items
2020-08-02 00:47:41,287 : INFO : sample=0.001 downsamples 87 most-common words
2020-08-02 00:47:41,287 : INFO : downsampling leaves estimated 5581 word corpus (40.8% of prior 13666)
2020-08-02 00:47:41,289 : INFO : estimated required memory for 169 words and 200 dimensions: 354900 bytes
2020-08-02 

2020-08-02 00:47:43,460 : INFO : sample=0.001 downsamples 172 most-common words
2020-08-02 00:47:43,461 : INFO : downsampling leaves estimated 11515 word corpus (83.5% of prior 13783)
2020-08-02 00:47:43,462 : INFO : estimated required memory for 374 words and 200 dimensions: 785400 bytes
2020-08-02 00:47:43,463 : INFO : updating layer weights
2020-08-02 00:47:43,473 : WARNING : Effective 'alpha' higher than previous training cycles
2020-08-02 00:47:43,474 : INFO : training model with 3 workers on 240 vocabulary and 200 features, using sg=0 hs=0 sample=0.001 negative=5 window=7
2020-08-02 00:47:43,487 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-02 00:47:43,488 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-02 00:47:43,489 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-02 00:47:43,491 : INFO : EPOCH - 1 : training on 14967 raw words (5874 effective words) took 0.0s, 521704 effective words/s
2020-08-02

2020-08-02 00:47:43,940 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-02 00:47:43,941 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-02 00:47:43,941 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-02 00:47:43,942 : INFO : EPOCH - 1 : training on 15234 raw words (5918 effective words) took 0.0s, 695627 effective words/s
2020-08-02 00:47:43,953 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-02 00:47:43,953 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-02 00:47:43,954 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-02 00:47:43,955 : INFO : EPOCH - 2 : training on 15234 raw words (6019 effective words) took 0.0s, 724396 effective words/s
2020-08-02 00:47:43,966 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-02 00:47:43,967 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-

2020-08-02 00:47:44,394 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-02 00:47:44,395 : INFO : EPOCH - 2 : training on 15311 raw words (6153 effective words) took 0.0s, 667219 effective words/s
2020-08-02 00:47:44,407 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-02 00:47:44,408 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-02 00:47:44,409 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-02 00:47:44,409 : INFO : EPOCH - 3 : training on 15311 raw words (6159 effective words) took 0.0s, 830249 effective words/s
2020-08-02 00:47:44,410 : INFO : training on a 45933 raw words (18444 effective words) took 0.0s, 451795 effective words/s
2020-08-02 00:47:44,412 : INFO : collecting all words and their counts
2020-08-02 00:47:44,413 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-08-02 00:47:44,415 : INFO : collected 940 word types from a corpus of 1531

2020-08-02 00:47:44,862 : INFO : training on a 42951 raw words (16552 effective words) took 0.0s, 421475 effective words/s
2020-08-02 00:47:44,862 : INFO : collecting all words and their counts
2020-08-02 00:47:44,863 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-08-02 00:47:44,865 : INFO : collected 853 word types from a corpus of 14317 raw words and 100 sentences
2020-08-02 00:47:44,866 : INFO : Updating model with new vocabulary
2020-08-02 00:47:44,867 : INFO : New added 158 unique words (15% of original 1011) and increased the count of 158 pre-existing words (15% of original 1011)
2020-08-02 00:47:44,869 : INFO : deleting the raw counts dictionary of 853 items
2020-08-02 00:47:44,870 : INFO : sample=0.001 downsamples 166 most-common words
2020-08-02 00:47:44,870 : INFO : downsampling leaves estimated 10720 word corpus (80.4% of prior 13331)
2020-08-02 00:47:44,872 : INFO : estimated required memory for 316 words and 200 dimensions: 663600 bytes
202

...

2020-08-02 00:54:33,728 : INFO : deleting the raw counts dictionary of 4812 items
2020-08-02 00:54:33,729 : INFO : sample=0.001 downsamples 152 most-common words
2020-08-02 00:54:33,730 : INFO : downsampling leaves estimated 61861 word corpus (143.6% of prior 43090)
2020-08-02 00:54:33,743 : INFO : estimated required memory for 2548 words and 200 dimensions: 5350800 bytes
2020-08-02 00:54:33,745 : INFO : updating layer weights
2020-08-02 00:54:33,751 : WARNING : Effective 'alpha' higher than previous training cycles
2020-08-02 00:54:33,752 : INFO : training model with 3 workers on 8589 vocabulary and 200 features, using sg=1 hs=0 sample=0.001 negative=5 window=7
2020-08-02 00:54:33,870 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-02 00:54:33,872 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-02 00:54:33,873 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-02 00:54:33,874 : INFO : EPOCH - 1 : training on

2020-08-02 00:54:35,708 : INFO : training model with 3 workers on 8596 vocabulary and 200 features, using sg=1 hs=0 sample=0.001 negative=5 window=7
2020-08-02 00:54:35,807 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-02 00:54:35,812 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-02 00:54:35,814 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-02 00:54:35,816 : INFO : EPOCH - 1 : training on 43031 raw words (30139 effective words) took 0.1s, 289383 effective words/s
2020-08-02 00:54:35,922 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-02 00:54:35,927 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-02 00:54:35,928 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-02 00:54:35,929 : INFO : EPOCH - 2 : training on 43031 raw words (30317 effective words) took 0.1s, 274211 effective words/s
2020-08-02 00:54:36,033 : INFO : worker

2020-08-02 00:54:37,816 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-02 00:54:37,818 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-02 00:54:37,820 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-02 00:54:37,820 : INFO : EPOCH - 2 : training on 44458 raw words (31364 effective words) took 0.1s, 277857 effective words/s
2020-08-02 00:54:37,935 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-02 00:54:37,937 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-02 00:54:37,939 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-02 00:54:37,940 : INFO : EPOCH - 3 : training on 44458 raw words (31528 effective words) took 0.1s, 273433 effective words/s
2020-08-02 00:54:37,941 : INFO : training on a 133374 raw words (94450 effective words) took 0.4s, 269582 effective words/s
2020-08-02 00:54:37,942 : INFO : collecting all words and their 

2020-08-02 00:54:39,827 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-02 00:54:39,827 : INFO : EPOCH - 3 : training on 45580 raw words (32638 effective words) took 0.1s, 284042 effective words/s
2020-08-02 00:54:39,828 : INFO : training on a 136740 raw words (98169 effective words) took 0.4s, 273537 effective words/s
2020-08-02 00:54:39,829 : INFO : collecting all words and their counts
2020-08-02 00:54:39,830 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-08-02 00:54:39,837 : INFO : collected 4266 word types from a corpus of 45580 raw words and 100 sentences
2020-08-02 00:54:39,838 : INFO : Updating model with new vocabulary
2020-08-02 00:54:39,842 : INFO : New added 1174 unique words (21% of original 5440) and increased the count of 1174 pre-existing words (21% of original 5440)
2020-08-02 00:54:39,849 : INFO : deleting the raw counts dictionary of 4266 items
2020-08-02 00:54:39,849 : INFO : sample=0.001 downsamples 170 mo

2020-08-02 00:54:41,458 : INFO : collected 3518 word types from a corpus of 27315 raw words and 60 sentences
2020-08-02 00:54:41,459 : INFO : Updating model with new vocabulary
2020-08-02 00:54:41,462 : INFO : New added 819 unique words (18% of original 4337) and increased the count of 819 pre-existing words (18% of original 4337)
2020-08-02 00:54:41,466 : INFO : deleting the raw counts dictionary of 3518 items
2020-08-02 00:54:41,467 : INFO : sample=0.001 downsamples 192 most-common words
2020-08-02 00:54:41,468 : INFO : downsampling leaves estimated 31499 word corpus (137.4% of prior 22918)
2020-08-02 00:54:41,557 : INFO : estimated required memory for 8609 words, 59531 buckets and 200 dimensions: 67673868 bytes
2020-08-02 00:54:41,559 : INFO : updating layer weights
2020-08-02 00:54:41,682 : INFO : Number of new ngrams is 17
2020-08-02 00:54:41,726 : WARNING : Effective 'alpha' higher than previous training cycles
2020-08-02 00:54:41,727 : INFO : training model with 3 workers on 860

In [29]:
for c, s in zip(cbow.wv.most_similar(['engineer']), skip_gram.wv.most_similar(['engineer'])):
    print(c, s)

('tester', 0.6862378120422363) ('devops', 0.5889719724655151)
('developer', 0.6647790670394897) ('tester', 0.5448418259620667)
('technician', 0.6424199342727661) ('fpga', 0.5426498651504517)
('programmer', 0.6294628381729126) ('opto', 0.5265384912490845)
('sr', 0.6243979334831238) ('atg', 0.5201519727706909)
('administrator', 0.6040023565292358) ('sqa', 0.5168964862823486)
('designer', 0.6038491725921631) ('murata', 0.5127512216567993)
('architect', 0.5957659482955933) ('mulesoft', 0.5110695958137512)
('scientist', 0.5900628566741943) ('routercim', 0.5072110891342163)
('analyst', 0.5836350917816162) ('algorithm', 0.5070369839668274)


/opt/conda/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


# Train occupation classifier

In [14]:
onet = Onet()
onet.print_summary_stats()

2020-08-02 16:41:28,196 : INFO : Manual build specified. Building O*NET CompetencyOntology via direct querying from O*NET site, or local cache.
2020-08-02 16:41:28,197 : INFO : Processing Content Model Reference
2020-08-02 16:41:29,778 : INFO : Processing occupation data
2020-08-02 16:41:31,138 : INFO : Processing Knowledge, Skills, Abilities
2020-08-02 16:42:03,946 : INFO : Processing tools and technology


Ontology summary statistics for onet
Num competencies: 32030
Num occupations: 1133
Num competency-occupation edges: 107305
Median occupations per competency: 1
Median competencies per occupation: 89
Mean occupations per competency: 3.350245090386837
Mean competencies per occupation: 94.70873786407768


In [38]:
class trainJobPostingParser(object):
    def __init__(self):
#         fname = '800_sample.json'
        fname = '8k_sample.json'
        f = open(fname, 'r')
        self.lines = f.read().split('\n')
        self.transformer = VirginiaTransformer(partner_id = 'VA')
        
    def __iter__(self) -> JobPostingGeneratorType:
        for line in self.lines:
            if line:
                yield self.transformer._transform(json.loads(line))

job_postings_train = trainJobPostingParser()

In [39]:
# target variable
full_soc = FullSOC(onet_cache=onet)

# design matrix
document_schema_fields = ['description','experienceRequirements', 'qualifications', 'skills']

dataset = itershuffle(job_postings_train)
train = islice(dataset, 0, 8000)
test =islice(dataset, 8000)

pipe_x = IterablePipeline(
    partial(nlp.fields_join, document_schema_fields=document_schema_fields),
    nlp.clean_str,
    nlp.word_tokenize,
    partial(nlp.vectorize, embedding_model=fasttext)
)

pipe_y = IterablePipeline(
    full_soc.transformer
)        


matrix = DesignMatrix(
    train, 
    full_soc,
    pipe_x,
    pipe_y,
)

# occupation classifier trainer
grid_config = {
                 'sklearn.ensemble.ExtraTreesClassifier': {
                     'n_estimators': [50, 100],
                     'criterion': ['entropy', 'gini'],
                     'max_depth': [20],
                     'max_features': ['log2'],
                     'min_samples_split': [10]
                      },
                 'sklearn.neural_network.MLPClassifier': {
                    'hidden_layer_sizes': [100, 500],
                     'activation': ['logistic', 'relu'],
                     'solver': ['adam']
                     },
                 }

cls_trainer = OccupationClassifierTrainer(
    matrix=matrix,
    k_folds=2,
    grid_config=grid_config,
    storage=FSStore('tmp/model_cache/soc_classifiers/monster'),
    n_jobs=14)


In [40]:
cls_trainer.train(save=False)



2020-08-02 11:19:07,879 : INFO : Start training 2020-08-02T11:18:59.715516
2020-08-02 11:19:07,881 : INFO : Building matrix
2020-08-02 11:19:33,997 : INFO : total jobpostings: 8000
2020-08-02 11:19:33,998 : INFO : filtered jobpostings: 8000
2020-08-02 11:19:33,998 : INFO : dropped jobposting: 0.0
2020-08-02 11:19:34,010 : INFO : training ExtraTreesClassifier
2020-08-02 11:19:34,011 : INFO : Creating model hash from unique data {'className': 'ExtraTreesClassifier', 'parameters': {'n_estimators': [50, 100], 'criterion': ['entropy', 'gini'], 'max_depth': [20], 'max_features': ['log2'], 'min_samples_split': [10]}, 'project_path': 'tmp/model_cache/soc_classifiers/monster', 'training_metadata': {'pipe_X': ['partial(func, *args, **keywords) - new function with partial application\n    of the given arguments and keywords.\n', '\n    Args:\n        text: A unicode string\n    Returns:\n        str: lowercased, sans punctuation, non-English letters\n    ', '\n    Args:\n        text (str): a uni

In [41]:
cls_trainer.best_estimators



[<ProxyObjectWithStorage at 0x7fe867213830 for GridSearchCV at 0x7fe880c3d198>,
 <ProxyObjectWithStorage at 0x7fe8672137d8 for GridSearchCV at 0x7fe880b0d208>]

## Evaluations

In [42]:
steps = [
    partial(nlp.fields_join, document_schema_fields=document_schema_fields),
    nlp.normalize,
    nlp.clean_str,
    nlp.word_tokenize,
]

evaluators = []
test_data = list(test)
for cls in cls_trainer.best_estimators:
    tester = OccupationClassifierTester(
        test_data_generator=job_postings_train, 
        preprocessing=steps, 
        classifier=CombinedClassifier(fasttext, cls)
    )
    evaluators.append(OnetOccupationClassificationEvaluator(tester))

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: generator 'itershuffle' raised StopIteration
  if __name__ == '__main__':


In [43]:
for e, c in zip(evaluators, cls_trainer.best_estimators):
    print(c.best_estimator_)
    print('accuracy: ', e.accuracy)
    print('precision: ', e.precision)
    print('f1: ', e.f1)
    print('major group: ', e.accuracy_major_group)
    print('macro precision: ', e.macro_precision)
    print('micro precision: ', e.micro_precision)
    print('recall: ', e.recall)
    print('macro recall: ', e.macro_recall)
    print('micro recall: ', e.micro_recall)
    print('macro f1: ', e.macro_f1)
    print('micro f1: ', e.micro_f1)
    print('\n')

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='entropy',
           max_depth=20, max_features='log2', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=10,
           min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=14,
           oob_score=False, random_state=None, verbose=0, warm_start=False)
accuracy:  1.0
precision:  [1. 1. 1. ... 1. 1. 1.]
f1:  [1. 1. 1. ... 1. 1. 1.]
major group:  1.0
macro precision:  1.0
micro precision:  1.0
recall:  [1. 1. 1. ... 1. 1. 1.]
macro recall:  1.0
micro recall:  1.0
macro f1:  1.0
micro f1:  1.0


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=500, learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shu

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Generate Job posting corpus for soc prediction

In [44]:
monster_220k.describe()

,country,country_code,date_added,has_expired,job_board,job_description,job_title,job_type,location,organization,page_url,salary,sector,uniq_id
count,22000,22000,122,22000,22000,22000,22000,20372,22000,15133,22000,3446,16806,22000
unique,1,1,77,1,1,18744,18749,39,8423,738,22000,1737,163,22000
top,United States of America,US,9/22/16,No,jobs.monster.com,12N Horizontal Construction Engineers Job Desc...,Monster,Full Time,"Dallas, TX",Healthcare Services,http://jobview.monster.com/Registered-Nurse-1-...,"40,000.00 - 100,000.00 $ /year",Experienced (Non-Manager),89706a078c37b0f70cec1dc93cff86db
freq,22000,22000,6,22000,22000,104,318,6757,646,1919,1,50,4594,1


In [45]:
# job_postings_generator = JobPostingCollectionSample()
corpus = CorpusCreator(job_postings)
corpus_creator = SimpleCorpusCreator()
common_classifier = CombinedClassifier(fasttext, cls)
# predicted_soc = common_classifier.predict_soc(corpus_creator._transform(sample_600))

soc_list = []
for job_posting in job_postings:
    predicted_soc = common_classifier.predict_soc(corpus_creator._transform(job_posting))
    soc_list.append(predicted_soc[0][0])

In [47]:
len(soc_list[1160:])

22000

# Merge with dataframe

In [48]:
print(monster_220k.columns)
monster_220k.head()

Index(['country', 'country_code', 'date_added', 'has_expired', 'job_board',
       'job_description', 'job_title', 'job_type', 'location', 'organization',
       'page_url', 'salary', 'sector', 'uniq_id'],
      dtype='object')


,country,country_code,date_added,has_expired,job_board,job_description,job_title,job_type,location,organization,page_url,salary,sector,uniq_id
0,United States of America,US,NaN,No,jobs.monster.com,TeamSoft is seeing an IT Support Specialist to...,IT Support Technician Job in Madison,Full Time Employee,"Madison, WI 53702",NaN,http://jobview.monster.com/it-support-technici...,NaN,IT/Software Development,11d599f229a80023d2f40e7c52cd941e
1,United States of America,US,NaN,No,jobs.monster.com,The Wisconsin State Journal is seeking a flexi...,Business Reporter/Editor Job in Madison,Full Time,"Madison, WI 53708",Printing and Publishing,http://jobview.monster.com/business-reporter-e...,NaN,NaN,e4cbb126dabf22159aff90223243ff2a
2,United States of America,US,NaN,No,jobs.monster.com,Report this job About the Job DePuy Synthes Co...,Johnson & Johnson Family of Companies Job Appl...,"Full Time, Employee",DePuy Synthes Companies is a member of Johnson...,Personal and Household Services,http://jobview.monster.com/senior-training-lea...,NaN,NaN,839106b353877fa3d896ffb9c1fe01c0
3,United States of America,US,NaN,No,jobs.monster.com,Why Join Altec? If you’re considering a career...,Engineer - Quality Job in Dixon,Full Time,"Dixon, CA",Altec Industries,http://jobview.monster.com/engineer-quality-jo...,NaN,Experienced (Non-Manager),58435fcab804439efdcaa7ecca0fd783
4,United States of America,US,NaN,No,jobs.monster.com,Position ID# 76162 # Positions 1 State CT C...,Shift Supervisor - Part-Time Job in Camphill,Full Time Employee,"Camphill, PA",Retail,http://jobview.monster.com/shift-supervisor-pa...,NaN,Project/Program Management,64d0272dc8496abfd9523a8df63c184c


In [49]:
col_toDrop = ['country', 'date_added', 'has_expired', 'job_board', 'salary', 'uniq_id', 'page_url']
monster_220k.drop(col_toDrop, axis=1, inplace=True)

In [50]:
monster_220k['soc_code'] = soc_list[1160:]

In [51]:
import pickle

dirPData = ''
f_name = dirPData + 'df_output.pickle'
with open(f_name, "wb") as f:
    pickle.dump(monster_220k, f)

In [52]:
len(list(set(soc_list)))

98

# Skills extraction

## Reopen and parse job postings data

In [9]:
dirPData = '../output/'
f_name = dirPData + 'df_output.pickle'

with open(f_name, "rb") as f:
    dict_ = pickle.load(f)

monster_220k = dict_
del f_name, dict_


In [12]:
monster_220k.head()

,country_code,job_description,job_title,job_type,location,organization,sector,soc_code,pattern,exact_match,soc_match,skills
0,US,TeamSoft is seeing an IT Support Specialist to...,IT Support Technician Job in Madison,Full Time Employee,"Madison, WI 53702",NaN,IT/Software Development,19-1099.00,None,None,None,None
1,US,The Wisconsin State Journal is seeking a flexi...,Business Reporter/Editor Job in Madison,Full Time,"Madison, WI 53708",Printing and Publishing,NaN,19-1099.00,None,None,None,None
2,US,Report this job About the Job DePuy Synthes Co...,Johnson & Johnson Family of Companies Job Appl...,"Full Time, Employee",DePuy Synthes Companies is a member of Johnson...,Personal and Household Services,NaN,17-2199.05,None,None,None,None
3,US,Why Join Altec? If you’re considering a career...,Engineer - Quality Job in Dixon,Full Time,"Dixon, CA",Altec Industries,Experienced (Non-Manager),11-9039.00,None,None,None,None
4,US,Position ID# 76162 # Positions 1 State CT C...,Shift Supervisor - Part-Time Job in Camphill,Full Time Employee,"Camphill, PA",Retail,Project/Program Management,15-1199.00,None,None,None,None


In [19]:
fname = '../data/50_sample.json'
f = open(fname, 'r')
data = f.read().split('\n')[0]
schema = json.loads(data)
del f, data, fname

In [20]:
df_json = []
for i in range(monster_220k.shape[0]):
    occupation = copy.deepcopy(schema)
    occupation['hiringOrganization']['location'] = monster_220k['location'][i]
    occupation['title'] = monster_220k['job_title'][i]
    occupation['jobDescription'] = monster_220k['job_description'][i].split('\n')
    occupation['normalizedTitle']['onetCode'] = monster_220k['soc_code'][i]
    df_json.append(occupation)
    
with open("../output/monster_soc_output.json", "w") as write_file:
    for i in df_json:
        json.dump(i, write_file)
        write_file.write('\n')

In [21]:
## reads json data file

JobPostingType = Dict[Text, Any]
JobPostingGeneratorType = Generator[JobPostingType, None, None]
MetadataType = Dict[Text, Dict[Text, Any]]

class monsterSOCOutputParser(object):
    def __init__(self):
        fname = '../output/monster_soc_output.json'
        f = open(fname, 'r')
        self.lines = f.read().split('\n')
        self.transformer = VirginiaTransformer(partner_id = 'VA')
        
    def __iter__(self) -> JobPostingGeneratorType:
        for line in self.lines:
            if line:
                yield self.transformer._transform(json.loads(line))
    
    @property
    def metadata(self) -> MetadataType:
        return {'job postings': {
            'downloaded_from': 'jobs.monster.com',
            'month': '2020-08',
            'purpose': 'monster_analysis'
        }}
    
job_postings = monsterSOCOutputParser()

In [22]:
monster_220k['pattern'] = None
monster_220k['exact_match'] = None
monster_220k['soc_match'] = None
monster_220k['skills'] = None

In [23]:
skill_extractor_p = SkillEndingPatternExtractor(only_bulleted_lines=False)
skill_extractor_e = ExactMatchSkillExtractor(onet.competency_framework)
skill_extractor_s = SocScopedExactMatchSkillExtractor(onet)

counter = 0

for job_posting in job_postings:
    
    pattern_dict = {}   
    em_dict = {}
    soc_dict = {}
    
    skills = []
    
    for candidate_skill in skill_extractor_p.candidate_skills(job_posting):
        pattern_dict[candidate_skill.skill_name] = candidate_skill.context
        skills.append(candidate_skill.skill_name)
    monster_220k['pattern'][counter] = pattern_dict
    
    for candidate_skill in skill_extractor_e.candidate_skills(job_posting):
        em_dict[candidate_skill.skill_name] = candidate_skill.context
        skills.append(candidate_skill.skill_name)
    monster_220k['exact_match'][counter] = em_dict
    
    for candidate_skill in skill_extractor_s.candidate_skills(job_posting):
        soc_dict[candidate_skill.skill_name] = candidate_skill.context
        skills.append(candidate_skill.skill_name)
    monster_220k['soc_match'][counter] = em_dict
    
    monster_220k['skills'][counter] = list(set(skills))
    counter += 1

2020-08-02 16:51:30,379 : INFO : Found 32029 entries for lookup
2020-08-02 16:51:31,582 : INFO : Found 32029 entries for lookup
2020-08-02 16:51:32,727 : INFO : Yielding candidate skill communication skills in context College degree preferred, 2 - 5 years experience in print and / or online advertising sales and be able to show consistent sales results in previous positions, Knowledge of the IT industry is preferred, Track record of creativity in sales approaches and solutions, Track record of successfully meeting and exceeding sales goals in media sales relevant to 1105 Medias line of business, Excellent client presentation and communication skills as well as strong customer service and organizational skills, The ideal candidate is energetic, self - motivated, team - oriented, and customer - centric, Understanding of how to research potential customers and use online analytics from a sales perspective, Weekly local travel to meet with clients / prospects is required, Minimal non local

2020-08-02 16:51:32,995 : INFO : Yielding candidate skill communication skills in context Inc. ( 6029) Job FunctionQuality ( Generalist) Report Previous Next {" Caption ":""," Images ":[{" Src ":" http :// images. monster. com / mm / xjjsonx / cjt1 / Carousel - 5. png "," Title ": null ," Text ": null ," Caption ": null },{" Src ":" http :// images. monster. com / mm / xjjsonx / cjt1 / Carousel - 1. png "," Title ": null ," Text ": null ," Caption ": null },{" Src ":" http :// images. monster. com / mm / xjjsonx / cjt1 / Carousel - 2. png "," Title ": null ," Text ": null ," Caption ": null },{" Src ":" http :// images. monster. com / mm / xjjsonx / cjt1 / Carousel - 3. png "," Title ": null ," Text ": null ," Caption ": null },{" Src ":" http :// images. monster. com / mm / xjjsonx / cjt1 / Carousel - 4. png "," Title ": null ," Text ": null ," Caption ": null}]} ‹ › × College degree preferred, 2 - 5 years experience in print and / or online advertising sales and be able to show consi

limit_output extension: Maximum message size of 10000 exceeded with 11659 characters

# Save dataframe

In [24]:
monster_220k.head()

,country_code,job_description,job_title,job_type,location,organization,sector,soc_code,pattern,exact_match,soc_match,skills
0,US,TeamSoft is seeing an IT Support Specialist to...,IT Support Technician Job in Madison,Full Time Employee,"Madison, WI 53702",NaN,IT/Software Development,19-1099.00,{'communication skills': 'College degree prefe...,{'call tracking software': 'Required Skills:• ...,{'call tracking software': 'Required Skills:• ...,"[microsoft office, organizational skills, call..."
1,US,The Wisconsin State Journal is seeking a flexi...,Business Reporter/Editor Job in Madison,Full Time,"Madison, WI 53708",Printing and Publishing,NaN,19-1099.00,{'communication skills': 'As part of your onli...,{'platforms': 'Candidates must have strong new...,{'platforms': 'Candidates must have strong new...,"[platforms, self, organizational skills, commu..."
2,US,Report this job About the Job DePuy Synthes Co...,Johnson & Johnson Family of Companies Job Appl...,"Full Time, Employee",DePuy Synthes Companies is a member of Johnson...,Personal and Household Services,NaN,17-2199.05,{'coaching skills': '• Strong leadership and c...,{'design': 'Directs staff in the design and ad...,{'design': 'Directs staff in the design and ad...,"[.• excellent communication skills, levels, co..."
3,US,Why Join Altec? If you’re considering a career...,Engineer - Quality Job in Dixon,Full Time,"Dixon, CA",Altec Industries,Experienced (Non-Manager),11-9039.00,{'verbal communication skills': 'MAJOR RESPONS...,{'design': 'MAJOR RESPONSIBILITIES:• Learns Al...,{'design': 'MAJOR RESPONSIBILITIES:• Learns Al...,"[rules, telecommunications, persuasion, skill,..."
4,US,Position ID# 76162 # Positions 1 State CT C...,Shift Supervisor - Part-Time Job in Camphill,Full Time Employee,"Camphill, PA",Retail,Project/Program Management,15-1199.00,{'communication skills': 'College degree prefe...,"{'self': 'College degree preferred, 2-5 years ...","{'self': 'College degree preferred, 2-5 years ...","[self, organizational skills, communication sk..."


In [25]:
dirPData = '../output/'
f_name = dirPData + 'df_output_skills.pickle'
with open(f_name, "wb") as f:
    pickle.dump(monster_220k, f)

In [26]:
# import pickle

# dirPData = ''
# f_name = dirPData + 'df_output.pickle'
# with open(f_name, "wb") as f:
#     pickle.dump(monster_220k, f)

dirPData = '../output/'
f_name = dirPData + 'df_output_skills.pickle'

with open(f_name, "rb") as f:
    dict_ = pickle.load(f)

df_output_skills = dict_
del f_name, dict_
    

In [29]:
df_output_skills.tail()

,country_code,job_description,job_title,job_type,location,organization,sector,soc_code,pattern,exact_match,soc_match,skills
21995,US,This is a major premier Cincinnati based finan...,Assistant Vice President - Controller Job in C...,Full Time,"Cincinnati, OH",NaN,NaN,19-1099.00,{'strong presentation skills': 'Proven strong ...,"{'monitors': 'Monitors costs and expenses, and...","{'monitors': 'Monitors costs and expenses, and...","[rules, monitoring, levels, organizational ski..."
21996,US,Luxury homebuilder in Cincinnati seeking multi...,Accountant Job in Cincinnati,Full Time,"Cincinnati, OH 45236",Construction - Residential & Commercial/Office,Manager (Manager/Supervisor of Staff),43-3099.00,{'communication skills': 'College degree prefe...,"{'self': 'College degree preferred, 2-5 years ...","{'self': 'College degree preferred, 2-5 years ...","[self, organizational skills, communication sk..."
21997,US,RE: Adobe AEM- Client - Loca...,AEM/CQ developer Job in Chicago,Full Time,"Chicago, IL 60602",NaN,NaN,17-2199.05,{'communication skills': 'Please send me resum...,{'self': 'Please send me resumes to vishakha@p...,{'self': 'Please send me resumes to vishakha@p...,"[self, organizational skills, communication sk..."
21998,US,Jernberg Industries was established in 1937 an...,Electrician - Experienced Forging Electrician ...,Full Time Employee,"Chicago, IL 60609","Jernberg Industries, Inc.",Installation/Maintenance/Repair,19-1029.00,{'diagnostic skills': 'We are seeking qualifie...,{'repairing': 'We are seeking qualified Candid...,{'repairing': 'We are seeking qualified Candid...,"[monitoring, troubleshooting, repairing, press..."
21999,US,Contract AdministratorCan you be the point per...,Contract Administrator Job in Cincinnati,Full Time,"Cincinnati, OH",NaN,Experienced (Non-Manager),19-1029.00,{'communication skills': 'College degree prefe...,"{'self': 'College degree preferred, 2-5 years ...","{'self': 'College degree preferred, 2-5 years ...","[self, organizational skills, communication sk..."
